In [ ]:
"""

pro reading_lightning

  ;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
  ; Reading data files from WWLLNN stations          
  ; Martinez C. Claudia - May 2016                    				
  ;                       Revised October 2017        
  ; Data provided by C. Rodgers (Otago U.)            
  ;                                                   
  ;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
  
  ;--------------------------------------
  ; Reading data from *.loc file
  ;--------------------------------------
  
  ;
  ; Data format
  ;
  ;  2013/01/01,00:00:00.072091, 14.1623,-110.1077, 22.0, 10
  ;
  ;  YEAR/MONTH/DAY, HOUR:MINUTE:SECOND, LATITUDE_N, LONGITUDE_E,
  ;  RMS_timing_error, # WWLLN stations contributing
  ;
  ;  The # WWLLN stations contributing should always be 5 or greater, and the RMS_timing_error should be less than or equal to 30 (which is a number in microseconds).
  
  ; Data format
  ;
  ;The format of each 1 day file in the zip file is:
  ;  year, month, day, hour, minute, second, lat, long, resid, Nsta, Energy, EnergyErr, NstaE
  ;
  ;  where:
  ;  date and time are in UT --> this is read as a string!!!
  ;  lat, long in fractional degrees
  ;  resid is residual fit error for location (in microseconds)
  ;  Nsta is the number of stations used in the location (>=5)
  ;  Energy is energy in joules of the radiated stroke energy during the 1.3 ms sample
  ;  EnergyErr if the fit error to the energy (also in joules)
  ;  NstaE is the number of stations with energy values used in the energy number (always <= Nsta)
  
  ; Lightning selection criteria
  ; 1. just take into account the number of stations (no energy factor)
  ;    criteria: contributing stations > 5, < 30 ms for timing error 
  ; 2. taking into account number of stations + energy of lightning strike detected
  ;    criteria: contributing stations > 5, < 30 ms for timing error, > 70% energy error
  
  ; Selection of the strikes that fill the requirements
  ; khan=0 for energy, khan=1 for counts
  ; user choses at terminal prompt if they run energy or counts 
  
  PRINT, 'Read lightning data for 2013'
  PRINT, 'Selection Criteria, 1 for counts, 0 for counts+energy' 
  READ, khan, PROMPT='Please write imput selection: '
; ii= months jj= days
; TIC TOC tracks how long code takes to run 

  FOR ii=1, 12 DO BEGIN ; 1 to 12
  
    FOR jj=1, 31 DO BEGIN ; 1 to 31
      TIC   
      IF khan EQ 1 THEN  BEGIN  
      ; Reading file for lightning counts
      file = 'I:\UCLA\WWLLN_Relocate_B_2013_daydata\A2013'+string(ii,'(i2.2)')+string(jj,'(i2.2)')+'.loc'
      openr, lun, file, /get_lun, error=err   
      
      IF (err EQ 0) THEN BEGIN      

        readcol, file, da, time, lat, lon, error, sta_nb, format='(a,a,f,f,f,f)', count=nb_l, /silent

        ; Selection of the strikes that fill the requirements
        ; khan=0 for energy, khan=1 for counts
        
        light_criteria, da, time, lat, lon, error, 0, sta_nb, 0, 0, 0, nb_l, khan
        
        ; Removing the NaN values to reduce space
        ; where there's not a NAN, do something
        ; NAN = not a number
        da = da[WHERE(~finite(da, /NAN), count)]
        time = time[WHERE(~finite(time, /NAN), count)] 
        lat = lat[WHERE(~finite(lat, /NAN), count)]
        lon = lon[WHERE(~finite(lon, /NAN), count)]
        
        ;lun = file? closing the file
        print, 'Lightning count done for '+string(da(1))
        close, lun
        free_lun, lun 
        
        ; adding strings to file name in terms of months and days
        save, /variables, filename='I:\UCLA\Revised\counts_2013'+string(ii,'(i2.2)')+string(jj,'(i2.2)')+'.sav'

      ENDIF ELSE print, 'ERROR!', file
      
      ENDIF 
      
      IF khan EQ 0 THEN BEGIN
        ; Reading file for lightning counts + energy
        ;2.2 is 2 digit format in IDL, in python its {2.2%} - means 2 digits 'string formatting' '%s or %d'
      file = 'I:\UCLA\AE_WWLLN_for_Claudia_2013\AE2013'+string(ii,'(i2.2)')+string(jj,'(i2.2)')+'.loc'
      openr, lun, file, /get_lun, error=err
      
      IF (err EQ 0) THEN BEGIN
      
        readcol, file, da, time, lat, lon, resid, sta_nb, nrj, nrj_err, sta_nb_nrj, format='(a,a,f,f,f,i,f,f,i)', count=nb_l, /silent

        ; Selection of the strikes that fill the requirements
        ; khan=0 for energy, khan=1 for counts
                
        light_criteria, da, time, lat, lon, 0, resid, sta_nb, nrj, nrj_err, sta_nb_nrj, nb_l, khan

        ; Removing the NaN values to reduce space
        da = da[WHERE(~finite(da, /NAN), count)]
        time = time[WHERE(~finite(time, /NAN), count)]
        lat = lat[WHERE(~finite(lat, /NAN), count)]
        lon = lon[WHERE(~finite(lon, /NAN), count)]
        nrj = nrj[WHERE(~finite(nrj, /NAN), count)]
        
        print, 'Lightning count + energy done for '+string(da(1))
        close, lun
        free_lun, lun
        
        save, /variables, filename='I:\UCLA\Revised\energy_counts_2013'+string(ii,'(i2.2)')+string(jj,'(i2.2)')+'.sav'
        
      ENDIF ELSE print, 'ERROR!', file
      
      ENDIF
      TOC
    ENDFOR
  ENDFOR
  
  
end





"""